# Introduction to PyTorch
[Jian Tao](https://coehpc.engr.tamu.edu/people/jian-tao/), Texas A&M University

Feb 12, 2020

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch

## 1. PyTorch as a fancy version of NumPy

* PyTorch supports Tensor computation (like NumPy, a fundamental package for scientific computing with Python) with strong GPU acceleration. You can find correspong numpy functions in PyTorch.
* It supports deep neural networks built on a tape-based autograd system

### NumPy examples

In [ ]:
x = np.array([[1,2,3],[3,4,5]], dtype=np.float64)
x.shape

In [ ]:
x = np.zeros([5,5])
x

In [ ]:
x = np.random.rand(5,5)
x

### PyTorch examples

In [ ]:
# create a 5X5 PyTorch tensor 
x = torch.zeros(5,5)
x

### Check CUDA support

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.device_count()

In [ ]:
dev = 'cuda:0' if torch.cuda.is_available() else 'cpu'
dev

In [ ]:
x = torch.rand((2,2), device=dev)

In [ ]:
x.device

In [ ]:
y = torch.rand(2,2).to(x.device)
y

In [ ]:
y.device

## 2. Autograd

The autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

In [ ]:
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)

In [ ]:
y = x + 2
print(y)

In [ ]:
print(y.grad_fn)

In [ ]:
z = y * y * 3
out = z.mean()

In [ ]:
z

In [ ]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))

# auto_grad is not activated by default
print(a.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

In [ ]:
b.backward()

In [ ]:
print(a)

## 3. Neural Networks

Neural networks can be constructed using the torch.nn package.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

### 3.1 Loss Function
There are several different loss functions under the nn package . A simple loss is: nn.MSELoss which computes the mean-squared error between the input and the target.

In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

In [ ]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]

### 3.2 Backprop

To backpropagate the error all we have to do is to loss.backward(). You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

### 3.3 Update the weights
The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

weight = weight - learning_rate * gradient

In [ ]:
# a simple implementation
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

### 3.4 Optimizer
However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

## 4. MNIST with Fully Connected Layers 

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt

In [ ]:
# Use standard FashionMNIST dataset
train = datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

In [ ]:
test = datasets.MNIST(root = './data/MNIST', train=False, download=True, 
                       transform=transforms.Compose([transforms.ToTensor()]))

In [ ]:
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=True)

In [ ]:
total = 0

In [ ]:
counter_dict={0:0, 1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}

In [ ]:
total = 0

for data in trainset:
    xs,ys =data
    for y in ys:
        counter_dict[int(y)] +=1
        total +=1
print (counter_dict)

In [ ]:
for i in counter_dict:
    print(f"{i}: {counter_dict[i]/total*100}")

In [ ]:
import seaborn as sns

sns.barplot(list(counter_dict.keys()), list(counter_dict.values()))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return F.log_softmax(x, dim=1)
        
net = Net()
print(net)

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
epochs =3
from tqdm import tqdm

In [ ]:
for epoch in tqdm(range(epochs)):
    for data in trainset:
        X,y = data
        net.zero_grad()
        output = net(X.view(-1, 28*28))
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
correct = 0 
total = 0
with torch.no_grad():
    for data in trainset:
        X, y =data
        output = net(X.view(-1, 784))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct +=1
            total +=1
print("Accuracy: ", round(correct/total,3))
    

In [ ]:
X

In [ ]:
plt.imshow(X[5].view(28,28))
print(torch.argmax(net(X[5].view(-1, 784))[0]))